In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import os
import time
import pickle
import json
import pandas as pd
from numba import jit
from types import SimpleNamespace
# from power_planner.data_reader import DataReader
from power_planner import graphs
from power_planner.utils.utils import (compute_pylon_dists)

In [ ]:
PATH_FILES = "../data"

# DEFINE CONFIGURATION
ID = "w_ksp_5"  # str(round(time.time() / 60))[-5:]

OUT_PATH = "outputs/path_" + ID
SCALE_PARAM = 2  # args.scale
# normal graph pipeline
# PIPELINE = [(2, 50), (1, 0)]  # [(1, 0)]  # [(4, 80), (2, 50), (1, 0)]  #
# random graph pipeline
PIPELINE = [(1, 0)]  # [(0.9, 40), (0, 0)]

GRAPH_TYPE = graphs.ImplicitLG
# LineGraph, WeightedGraph, RandomWeightedGraph, RandomLineGraph, PowerBF
# TwoPowerBF, WeightedKSP
print("graph type:", GRAPH_TYPE)
# summarize: mean/max/min, remove: all/surrounding, sample: simple/watershed
NOTES = "None"  # "mean-all-simple"

IOPATH = os.path.join(PATH_FILES, "de_data_1_" + str(SCALE_PARAM) + ".dat")
    
# READ DATA
with open(IOPATH, "rb") as infile:
    data = pickle.load(infile)
    (instance, edge_inst, instance_corr, config) = data

In [ ]:
graph = GRAPH_TYPE(instance, instance_corr, edge_inst)

In [ ]:
path, path_costs, cost_sum = graph.single_sp(**vars(config.graph))

In [ ]:
graph.graph.get_shortest_path(graph.start_inds, graph.dest_inds)

In [ ]:
path, path_costs, cost_sum = graph.sp_trees(**vars(config.graph))

In [ ]:
path = np.asarray(path)
plt.figure(figsize=(10,10))
plt.imshow(graph.pos2node)
plt.plot(path[:,1], path[:,0])
plt.show()

In [ ]:

from power_planner.ksp import KSP

In [ ]:
ksp = KSP(graph)
out_ksp = ksp.laplace(5)

In [ ]:
# from power_planner.alternative_paths import AlternativePaths
alt = AlternativePaths(graph)
# rep_path,_,_ = alt.replace_single_edge(120,  56, 125,  56)
window_path, _, _ = alt.replace_window(200, 250,180, 200)

In [ ]:
plt.imshow(graph.angle_cost_array)
plt.show()

In [ ]:
# window_path = rep_path
window_path = np.asarray(window_path)
path_orig = np.asarray(path)
plt.plot(window_path[:,0], window_path[:,1])
plt.plot(path_orig[:,0], path_orig[:,1])

In [ ]:
config.graph

### Evaluation of outputs

In [ ]:
with open("../../outputs/power analysis/power_analysis_belgium_paths.dat", "rb") as infile:
    #power analysis/power_analysis_belgium_paths.dat
    paths = pickle.load(infile)

In [ ]:
import seaborn as sb

In [ ]:
for p in paths:
    print("mean:", round(np.mean(p), 2), "sum:", round(np.sum(p), 2))
    sb.distplot(p)
    plt.ylim(0,0.4)
    plt.show()

## Create random instance for Github

In [ ]:
test = instance_corr.copy()
test[6:245, :60]=1
plt.imshow(test)

In [ ]:
[1] + [round(c,1) for c in np.logspace(0.1, 0.6, 6)]

In [ ]:
random_inst = np.random.rand(*instance.shape)

In [ ]:
random_inst.shape

In [ ]:
random_edge_inst = random_inst + (np.random.rand(*instance.shape)-0.5)*0.1

In [ ]:
random_inst[2,3,3]

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
smooth = np.asarray([gaussian_filter(random_inst[i], 5) for i in range(3)])

In [ ]:
plt.imshow(smooth[0])
plt.colorbar()

In [ ]:
smooth_edge = np.asarray([gaussian_filter(random_edge_inst[i], 5) for i in range(3)])

In [ ]:
smooth = (smooth-np.min(smooth))/(np.max(smooth)-np.min(smooth))

In [ ]:
smooth_edge = (smooth_edge-np.min(smooth_edge))/(np.max(smooth_edge)-np.min(smooth_edge))

In [ ]:
data_out = (smooth, smooth_edge, test, config)
with open("../data/test_data_1_2.dat", "wb") as outfile:
    pickle.dump(data_out, outfile)
print("successfully saved data")

In [ ]:
config.graph.scale

### Check average cost path - did running average work?

In [ ]:
nonavg = pd.read_csv("../../outputs/test_nonavg_ch_coords.csv_0.csv")
avg = pd.read_csv("../../outputs/test_avg_ch_coords.csv_0.csv")

In [ ]:
avg_path = (np.asarray(avg[["X_raw", "Y_raw"]])/5).astype(int)
nonavg_path = (np.asarray(nonavg[["X_raw", "Y_raw"]])/5).astype(int)

In [ ]:
costs1 = [graph.instance[tuple(p)] for p in avg_path]
costs2 = [graph.instance[tuple(p)] for p in nonavg_path]

In [ ]:
np.sum(costs1), np.mean(costs1), np.sum(costs2), np.mean(costs2)

## Check De output

In [ ]:
json_path = "../../outputs/testtt_2_de_0.csv" # de_inst_ksp/de_inst_2_e5_d30/test_2_e5_d30de_4.csv"

In [ ]:
path = pd.read_csv(json_path)[["X_raw", "Y_raw"]]

In [ ]:
path = np.asarray(path)

In [ ]:
pd.read_csv(json_path)

In [ ]:
from power_planner.utils.utils_costs import CostUtils
from power_planner.utils.utils import bresenham_line

In [ ]:
def compute_edge_costs(path, instance):
    e_costs = []
    for p in range(len(path) - 1):
        point_list = bresenham_line(
            path[p][0], path[p][1], path[p + 1][0], path[p + 1][1]
        )
        print([instance[i, j] for (i, j) in point_list[1:-1]])
        e_costs.append(
            np.mean([instance[i, j] for (i, j) in point_list[1:-1]])
        )
    # to make it the same size as other costs
    e_costs.append(0)
    return e_costs

In [ ]:
dirty_extend = edge_inst_new.copy()
x_len, y_len = edge_inst_new.shape
for i in range(1, x_len - 1):
    for j in range(1, y_len - 1):
        if np.any(edge_inst_new[i - 1:i + 2, j - 1:j + 2] == np.inf):
            dirty_extend[i, j] = np.inf

In [ ]:
np.all(dirty_extend==edge_inst_new)

In [ ]:
plt.imshow(dirty_extend[640:670, 550:680])
plt.show()

In [ ]:
edge_inst_new = np.sum(
            np.moveaxis(edge_inst, 0, -1) *
            config.graph.class_weights,
            axis=2
        )
print(compute_edge_costs((path/SCALE_PARAM).astype(int), edge_inst_new))

In [ ]:
path_scaled = (path/SCALE_PARAM).astype(int)
for p in range(len(path_scaled) - 1):
    point_list = bresenham_line(
        path_scaled[p][0], path_scaled[p][1], path_scaled[p + 1][0], path_scaled[p + 1][1]
    )
    for (i,j) in point_list:
        if edge_inst_new[i,j]==np.inf:
            pass
    print(point_list)

In [ ]:
plt.imshow(edge_inst_new[640:670, 550:680])

In [ ]:
print(path_scaled[-10:])

In [ ]:
import resource

In [ ]:
print(
                "Cannot initialize KSP object with a graph without"\
                "shortest path trees in both directions!"
            )

In [ ]:
a = np.random.rand(4,2)

In [ ]:
a

In [ ]:
np.flip(a, axis=0)

## Improve memory usage by efficient stack
only the incoming edges of the stack will be updated--> only put them in memory

In [ ]:
stack_array = np.asarray(stack)
dists_new = np.concatenate((stack_array, np.zeros((len(stack), n_neighbors))+np.inf), axis=1)

In [ ]:
pos2node = (np.zeros(self.instance.shape)-1).astype(int) # -1 for the unfilled ones
# make mapping to position
for i in range(len(stack_array)):
    (x,y) = tuple(stack_array[i])
    pos2node[x,y] = i

In [ ]:
inst_x_len, inst_y_len = instance.shape
for i in range(len(stack_array)):
    v_x = stack[i,0]
    v_y = stack[i,1]
    for s in range(len(shifts)):
        neigh_x = v_x + shifts[s][0]
        neigh_y = v_y + shifts[s][1]
        if (
            0 <= neigh_x < inst_x_len and 0 <= neigh_y < inst_y_len
            and pos2node[neigh_x, neigh_y]>=0
        ):
            neigh_stack_ind = pos2node[neigh_x, neigh_y]
            # add up pylon cost + angle cost + edge cost
            cost_per_angle = stack[i, 2:] + angles_all[s] + instance[
                neigh_x, neigh_y] + edge_cost[neigh_stack_ind, s+2]
            # update distances and predecessors
            stack[neigh_stack_ind,s+2] = np.min(cost_per_angle)
            preds[neigh_stack_ind, s+2] = np.argmin(cost_per_angle)

# Test angle update algorithm

In [ ]:
from power_planner.graphs.fast_shortest_path import sp_dag, efficient_update_sp
from power_planner.utils.utils import angle, discrete_angle_costs, angle_360

### Test data

In [ ]:
from types import SimpleNamespace
from power_planner import graphs
from power_planner.utils.utils import bresenham_line
expl_shape = (50, 50)
# create configuration
cfg = SimpleNamespace()
cfg.PYLON_DIST_MIN = 3
cfg.PYLON_DIST_MAX = 5
start_inds = np.array([6, 6])
dest_inds = np.array([41, 43])
cfg.start_inds = start_inds
cfg.dest_inds = dest_inds
cfg.ANGLE_WEIGHT = 0.25
cfg.EDGE_WEIGHT = 0
cfg.MAX_ANGLE = np.pi / 2
cfg.MAX_ANGLE_LG = np.pi / 4
cfg.layer_classes = ["dummy_class"]
cfg.class_weights = [1]

# construct simple line instance
example_inst = np.ones(expl_shape)
# construct corresponding corridor
working_expl_corr = np.zeros(expl_shape)
line = bresenham_line(
    start_inds[0], start_inds[1], dest_inds[0], dest_inds[1]
)
for (i, j) in line:
    working_expl_corr[i - 1:i + 1, j - 1:j + 1] = 1

# construct instance that required 90 degree angle
high_angle_corr = np.zeros(expl_shape)
high_angle_corr[start_inds[0], start_inds[1]:dest_inds[1] - 3] = 1
high_angle_corr[start_inds[0], dest_inds[1]] = 1
high_angle_corr[start_inds[0] + 3:dest_inds[0] + 1, dest_inds[1]] = 1

graph = graphs.ImplicitLG(
            np.array([example_inst]),
            working_expl_corr,
            n_iters=10,
            verbose=0
        )

In [ ]:
def single_sp(self, power=1, **kwargs):
    """
    Function for full processing until shortest path
    """
    self.start_inds = kwargs["start_inds"]
    self.dest_inds = kwargs["dest_inds"]
    self.set_shift(
        kwargs["PYLON_DIST_MIN"],
        kwargs["PYLON_DIST_MAX"],
        self.dest_inds - self.start_inds,
        kwargs["MAX_ANGLE"],
        max_angle_lg=kwargs["MAX_ANGLE_LG"]
    )
    print("1) Initialize shifts and instance (corridor)")
    self.set_edge_costs(
        kwargs["layer_classes"],
        kwargs["class_weights"],
        angle_weight=kwargs["ANGLE_WEIGHT"]
    )
    self.instance = self.instance**power
    # add vertices
    self.add_nodes()
    return graph

# config.graph.MAX_ANGLE = 3.14
graph = single_sp(graph, **vars(config.graph)) # fg)) #

In [ ]:
def _precompute_angles(self):
    tic = time.time()
    angles_all = np.zeros((len(self.shifts), len(self.shifts)))
    angles_all += np.inf
    for i in range(len(self.shifts)):
        for j, s in enumerate(self.shifts):
            ang = angle(s, self.shifts[i])
            # if ang <= self.angle_norm_factor:
            angles_all[i, j] = discrete_angle_costs(
                ang, self.angle_norm_factor
            )
    self.time_logs["compute_angles"] = round(time.time() - tic, 3)
    # multiply with angle weights, need to prevent that not inf * 0
   # angles_all[angles_all < np.inf
    #           ] = angles_all[angles_all < np.inf] * self.angle_weight
    return angles_all

graph.angle_cost_array = _precompute_angles(graph)

In [ ]:
shift_angles = [angle(s, graph.dest_inds-graph.start_inds) for s in graph.shifts]

In [ ]:
prob_vecs = [graph.shifts[j] for j in range(140, 155)]

In [ ]:
working = graph.dest_inds-graph.start_inds
not_work = graph.shifts[0]

In [ ]:
plt.plot([angle_360(v, working) for v in prob_vecs])
print([angle_360(v, not_work) for v in prob_vecs])
plt.plot([angle_360(v, not_work) for v in prob_vecs])

In [ ]:
not_work = np.asarray(not_work)
for vec1 in prob_vecs:
    vec1 = np.asarray(vec1)
    x1, y2 = vec1 / np.linalg.norm(vec1)
    y1, y2 = not_work/np.linalg.norm(not_work)
    dot = x1 * x2 + y1 * y2  # dot product
    det = x1 * y2 - y1 * x2  # determinant
    angle = np.arctan2(det, dot)
    print(angle)

In [ ]:
plt.plot(graph.angle_cost_array[40])
plt.show()

In [ ]:
dists = graph.dists.copy()
preds = graph.preds.copy()
shifts = graph.shifts
stack = graph.stack_array.copy()
pos2node = graph.pos2node.copy()
angles_all = graph.angle_cost_array
instance = graph.instance
edge_cost = np.zeros(graph.dists.shape)

In [ ]:
dists = graph.dists.copy()
preds = graph.preds.copy()
shifts = graph.shifts
stack = graph.stack_array.copy()
pos2node = graph.pos2node.copy()
angles_all = graph.angle_cost_array
instance = graph.instance
edge_cost = np.zeros(graph.dists.shape)
@jit(nopython=True)
def efficient_update_sp(dists, preds, shifts, stack, pos2node, angles_all, instance, edge_cost):
    inst_x_len, inst_y_len = instance.shape
    n_neighbors = len(shifts)
    for i in range(len(dists)):
        v_x = stack[i, 0]
        v_y = stack[i, 1]

        # sort the in edge distances and initialize
        initial_S = np.argsort(dists[i])
        marked_plus = np.zeros(n_neighbors)
        marked_minus = np.zeros(n_neighbors)

        # initialize dists and do first pass
        neighbor_vals = np.zeros(n_neighbors)+np.inf
        neighbor_inds = np.zeros(n_neighbors) -1
        ground_truth = np.zeros(n_neighbors)+np.inf
        ground_truth_pred = np.zeros(n_neighbors)

        for s in range(n_neighbors):
            neigh_x = int(v_x + shifts[s][0])
            neigh_y = int(v_y + shifts[s][1])
            if (
                    0 <= neigh_x < inst_x_len and 0 <= neigh_y < inst_y_len
                    and pos2node[neigh_x, neigh_y] >= 0 and instance[neigh_x, neigh_y]<np.inf
                ):            
                # PROBLE
                neighbor_vals[s] = instance[neigh_x, neigh_y]
                neigh_stack_ind = pos2node[neigh_x, neigh_y]
                neighbor_inds[s] = neigh_stack_ind
                # initialize distances to the straight line value
                dists[neigh_stack_ind, s] = dists[i, s] + instance[neigh_x, neigh_y]+ edge_cost[neigh_stack_ind, s]
                preds[neigh_stack_ind, s] = s

                cost_per_angle = dists[i] + angles_all[s] + instance[
                            neigh_x, neigh_y] + edge_cost[neigh_stack_ind, s]
                ground_truth[s] = np.min(cost_per_angle)
                ground_truth_pred[s] = np.argmin(cost_per_angle)
        # if i==67:
        #     print(ground_truth_pred)
        #     print(neighbor_inds)
        #     print(initial_S)

        # set current tuple: in edge and shift (out edge index unncessary because same as in edge)
        current_in_edge = initial_S[0]
        current_shift = 0
        tuple_counter = 0

        # debug
        update_shift = np.zeros(n_neighbors)

        while tuple_counter<len(initial_S)-1:
            # best out edge is exactly the same shift!
            current_out_edge = (current_in_edge+current_shift)%n_neighbors
            # if current_out_edge>=n_neighbors:
            #     current_shift = -1
            #     continue
            # # next tuple 
            # elif current_out_edge<0:
            #     current_shift = 0
            #     tuple_counter+=1
            #     current_in_edge = initial_S[tuple_counter]
            #     continue
            # print(current_out_edge, current_shift)
            # compute possible update value:
            update_val = dists[i, current_in_edge] + angles_all[current_out_edge, current_in_edge]

            if current_shift==0:
                marked = marked_plus[current_out_edge] and marked_minus[current_out_edge]
            elif current_shift>0:
                marked = marked_plus[current_out_edge]
            else:
                marked = marked_minus[current_out_edge]
            # update only if better
            neigh_stack_ind = int(neighbor_inds[current_out_edge])
            # if i==71 and (current_in_edge==148 or current_in_edge==145):
            #     print("SHIFT", current_shift, "marked", marked, "neigh_stack_ind", neigh_stack_ind)
            #     print("update with", update_val + neighbor_vals[current_out_edge] + edge_cost[neigh_stack_ind, current_out_edge])
            #     print("current dist", dists[neigh_stack_ind, current_out_edge])
            #     print(np.around(update_val + neighbor_vals[current_out_edge] + edge_cost[neigh_stack_ind, current_out_edge],5) <= np.around(dists[neigh_stack_ind, current_out_edge], 5))
            #     print("second comparison", np.around(update_val, 5), np.around(dists[i, current_out_edge], 5))
            # print(marked, neigh_stack_ind, update_val)
            # actual update: only if the neighbor exists
            # PROBLEM: what if angle cost becomes inf
            if marked==0 and neigh_stack_ind>=0 and np.around(update_val + neighbor_vals[current_out_edge] + edge_cost[neigh_stack_ind, current_out_edge],5) <= np.around(dists[neigh_stack_ind, current_out_edge], 5):
                dists[neigh_stack_ind, current_out_edge] = update_val + neighbor_vals[current_out_edge] + edge_cost[neigh_stack_ind, current_out_edge]
                preds[neigh_stack_ind, current_out_edge] = current_in_edge
                update_shift[current_out_edge] = current_shift
                progress_one = True
                
            # inf neighbor --> jump over it if its incoming edge is worse
            elif marked==0 and neigh_stack_ind<0 and np.around(update_val, 5) <= np.around(dists[i, current_out_edge], 5):
                progress_one = True

            # already marked or update not successful:
            # Consider first edge in other direction or next overall tuple
            else:
                progress_one = False
                if current_shift>0:
                    current_shift = -1
                else:
                    # get next tuple from stack
                    tuple_counter+=1
                    current_in_edge = initial_S[tuple_counter]
                    current_shift = 0
                        # Progress to next edge
        
            if progress_one:
                    
                if current_shift < 0:
                    current_shift -= 1
                if current_shift <= 0:
                    marked_minus[current_out_edge] = 1
                if current_shift >= 0:
                    current_shift += 1
                    marked_plus[current_out_edge] = 1
                    
                    
        # CHECK GT
        # for s in range(n_neighbors):
        #     stack_ind = int(neighbor_inds[s])
        #     if stack_ind>=0 and not np.isclose(dists[stack_ind, s], ground_truth[s]):
        #         print("PROBLEM")
        #         # print(dists[i], initial_S[0]) # , ground_truth[s])
        #         # neigh_x = int(v_x + shifts[s][0])
        #         # neigh_y = int(v_y + shifts[s][1])
        #         print(s)
        #         print("updated with", update_shift[s])
        #         print(dists[stack_ind, s], ground_truth[s])
        #         print("new pred", preds[stack_ind, s], "gt_pred", ground_truth_pred[s])
        #         print(dists[i, int(preds[stack_ind, s])], dists[i, int(ground_truth_pred[s])])
        #         print("----------")
        #         # print(dists[i])
        #         # print(instance[neigh_x, neigh_y])
        # # print("-------------------------", i)
    return dists, preds

dists, preds = efficient_update_sp(dists.copy(), preds.copy(), shifts, stack, pos2node, angles_all, instance, edge_cost)      

### Display and check if equivalent

In [ ]:
def display_dists(self, edge_array, func=np.min):
    arr = np.zeros(self.pos2node.shape)
    for i in range(len(self.pos2node)):
        for j in range(len(self.pos2node[0])):
            ind = self.pos2node[i, j]
            if ind >= 0:
                arr[i, j] = func(edge_array[ind, :])
    plt.imshow(arr)
    plt.colorbar()
    plt.show()
    return arr

In [ ]:
tic = time.time()
dists1, preds1 = efficient_update_sp(stack, pos2node, shifts, angles_all, graph.dists.copy(), graph.preds.copy(), instance, edge_cost)
print(time.time()-tic)


In [ ]:
arr = display_dists(graph, dists)

In [ ]:
tic = time.time()
dists2, preds2 = sp_dag(stack, pos2node, shifts, angles_all, graph.dists.copy(), graph.preds.copy(), instance, edge_cost)
print(time.time()-tic)


In [ ]:
arr2 = display_dists(graph, dists2)

In [ ]:
arr_copy = arr.copy()
arr_copy[arr_copy==np.inf] = 0
arr2_copy = arr2.copy()
arr2_copy[arr2_copy==np.inf] = 0
plt.figure(figsize=(20,20))
plt.imshow(arr_copy-arr2_copy)
plt.colorbar()

In [ ]:
len(np.where(arr!=arr2)[0])

In [ ]:
arr.shape

In [ ]:
arr2[tuple(graph.dest_inds)], arr[tuple(graph.dest_inds)]

In [ ]:
np.where(arr!=arr2)